<a href="https://colab.research.google.com/github/Soumadeep555/RAG/blob/main/Multimodal_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install the required libraries:

In [ ]:
! pip install "unstructured[all-docs]" pillow pydantic lxml matplotlib

In [ ]:
!sudo apt-get update

In [ ]:
!sudo apt-get install poppler-utils

In [ ]:
!sudo apt-get install tesseract-ocr

In [ ]:
!sudo apt-get install libtesseract-dev

In [ ]:
!pip install unstructured-pytesseract

In [ ]:
!pip install langchain_google_genai

## Data Parsing / Data Extraction:

In [ ]:
from unstructured.partition.pdf import partition_pdf

In [ ]:
raw_pdf_elements=partition_pdf(
    filename="/content/drive/MyDrive/Generative_AI_Advanced_NLP/Advance_NLP_Generative_AI/CODE/data/rag-for-nlp.pdf",
    strategy="hi_res",
    extract_images_in_pdf=True,
    extract_image_block_types=["Image","Table"],
    extract_image_block_to_payload=False,
    extract_image_block_output_dir="/content/drive/MyDrive/Generative_AI_Advanced_NLP/Advance_NLP_Generative_AI/CODE/data/extracted_data_1"
    )

In [ ]:
raw_pdf_elements

In [ ]:
Header=[]
Footer=[]
Title=[]
NarrativeText=[]
Text=[]
ListItem=[]
Image=[]
Table=[]

for element in raw_pdf_elements:
  if "unstructured.documents.elements.Header" in str(type(element)):
            Header.append(str(element))
  elif "unstructured.documents.elements.Footer" in str(type(element)):
            Footer.append(str(element))
  elif "unstructured.documents.elements.Title" in str(type(element)):
            Title.append(str(element))
  elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
            NarrativeText.append(str(element))
  elif "unstructured.documents.elements.Text" in str(type(element)):
            Text.append(str(element))
  elif "unstructured.documents.elements.ListItem" in str(type(element)):
            ListItem.append(str(element))
  elif "unstructured.documents.elements.Image" in str(type(element)):
            Image.append(str(element))
  elif "unstructured.documents.elements.Table" in str(type(element)):
            Table.append(str(element))

In [ ]:
Header

In [ ]:
Footer

In [ ]:
ListItem

In [ ]:
Title

In [ ]:
NarrativeText

In [ ]:
NarrativeText[0]

In [ ]:
NarrativeText[1]

In [ ]:
Text

In [ ]:
Table

In [ ]:
Image

## Load the Model and Embedding Model:

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [ ]:
import os
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
def load_model(model_name):
    if model_name == "gemini-pro": #To deal with Text
        return ChatGoogleGenerativeAI(model="gemini-1.5-pro")
    elif model_name == "gemini-1.5-flash": #To deal with Images
        return ChatGoogleGenerativeAI(model="gemini-1.5-flash")
    elif model_name == "embedding": #To generate Embeddings
        return GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    else:
        raise ValueError(f"Unsupported model name: {model_name}")

In [ ]:
model=load_model("gemini-1.5-flash")

In [ ]:
model_text=load_model("gemini-pro")

In [ ]:
#model_text.invoke("hi")

#### Exceeded the limit for Google Gemini, so proceeding with OpenAI:

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OpenAI_API_Key')

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
!pip install langchain-openai

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

In [ ]:
def load_model(model_name):
    if model_name == "chat": # To deal with Text
        return ChatOpenAI(model="gpt-4o")
    elif model_name == "vision": # To deal with Images
        return ChatOpenAI(model="gpt-4o") # gpt-4o supports vision
    elif model_name == "embedding": # To generate Embeddings
        return OpenAIEmbeddings(model="text-embedding-3-small")
    else:
        raise ValueError(f"Unsupported model name: {model_name}")

In [ ]:
model = load_model("vision")

In [ ]:
model_text = load_model("chat")

In [ ]:
model_text.invoke("hi")

# DATA INGESTION:

## Prepare the data for Vector Database:

## Summary of the Text:

In [ ]:
!pip install langchain_groq

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt_text="""You are an assistant tasked with summarizing text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text elements. \
    Give a concise summary of the table or text that is well optimized for retrieval.text: {element} """

In [ ]:
prompt=ChatPromptTemplate.from_template(prompt_text)

In [ ]:
#model=load_model("gemini-pro")

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [ ]:
groq_model=ChatGroq(model="deepseek-r1-distill-llama-70b",api_key=GROQ_API_KEY)

In [ ]:
summarize_chain = {"element": lambda x: x} |prompt| groq_model | StrOutputParser()

In [ ]:
type(NarrativeText)

In [ ]:
len(NarrativeText)

In [ ]:
NarrativeText=NarrativeText[:10]

In [ ]:
len(NarrativeText)

In [ ]:
text_summary=[]
text_summary=summarize_chain.batch(NarrativeText,{"max_concurrency": 5})

In [ ]:
text_summary

In [ ]:
print(text_summary[4])

In [ ]:
import re
clean_text = re.sub(r"<think>.*?</think>\s*", "", text_summary[4], flags=re.DOTALL)

In [ ]:
clean_text

#### Summary of the Table:

In [ ]:
prompt_text = """You are an AI Assistant tasked with summarizing tables for retrieval. \
    These summaries will be embedded and used to retrieve the raw table elements. \
    Give a concise summary of the table that is well optimized for retrieval. Table:{element} """

In [ ]:
prompt = ChatPromptTemplate.from_template(prompt_text)

In [ ]:
summarize_chain = {"element": lambda x: x} | prompt | groq_model | StrOutputParser()

In [ ]:
table_summaries = []
table_summaries = summarize_chain.batch(Table, {"max_concurrency": 5})

In [ ]:
print(table_summaries[0])

#### Summary of the Images:

In [ ]:
import base64
import os
from langchain_core.messages import AIMessage, HumanMessage

In [ ]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:
image_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [ ]:
image_model.invoke("hi")

In [ ]:
def image_summarize(img_base64,prompt):
    chat = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
    msg = chat.invoke(
        [
            HumanMessage(
                content=[
                    {
                    "type": "text",
                     "text": prompt
                     }
                    ,
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                    },
                ]
            )
        ]
    )
    return msg.content

In [ ]:
def generate_img_summaries(path):
    """
    Generate summaries and base64 encoded strings for images
    path: Path to list of .jpg files extracted by Unstructured
    """

    # Store base64 encoded images
    img_base64_list = []

    # Store image summaries
    image_summaries = []

    # Prompt
    prompt = """You are an assistant tasked with summarizing images for retrieval. \
    These summaries will be embedded and used to retrieve the raw image. \
    Give a concise summary of the image that is well optimized for retrieval."""

    # Apply to images
    for img_file in sorted(os.listdir(path)):
        if img_file.endswith(".jpg"):
            img_path = os.path.join(path, img_file)
            base64_image = encode_image(img_path)
            img_base64_list.append(base64_image)
            image_summaries.append(image_summarize(base64_image, prompt))


    return img_base64_list, image_summaries

In [ ]:
fpath="/content/drive/MyDrive/Generative_AI_Advanced_NLP/Advance_NLP_Generative_AI/CODE/data/extracted_data_1"

In [ ]:
img_base64_list, image_summaries=generate_img_summaries(fpath)

In [ ]:
img_base64_list

In [ ]:
image_summaries

In [ ]:
image_summaries[0]

# Store the Data inside the Vector Database and Create DATA RETRIEVER:

## VectorDatabase -> ChromaDB -> In Memory:

In [ ]:
!pip install -U langchain-community

In [ ]:
import uuid
from langchain_core.documents import Document
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [ ]:
def create_multi_vector_retriever(vectorstore, text_summaries, texts, table_summaries, tables, image_summaries, images):
  store=InMemoryStore()
  id_key="doc_id"

  retriever=MultiVectorRetriever(
      vectorstore=vectorstore,
      docstore=store,
      id_key=id_key,
  )

  def add_documents(retriever, doc_summaries, doc_contents):

      doc_ids = [str(uuid.uuid4()) for _ in doc_contents]

      summary_docs = [
              Document(page_content=summary, metadata={id_key: doc_ids[i]})

              for i, summary in enumerate(doc_summaries)
          ]

      retriever.vectorstore.add_documents(summary_docs)
      retriever.docstore.mset(list(zip(doc_ids, doc_contents)))


  if text_summaries:
        add_documents(retriever, text_summaries, texts)
  if table_summaries:
        add_documents(retriever, table_summaries, tables)
  if image_summaries:
        add_documents(retriever, image_summaries, images)

  return retriever



In [ ]:
!pip install chromadb

In [ ]:
embedding_model=load_model("embedding")

In [ ]:
vectorstore=Chroma(collection_name="MMRAG",embedding_function=embedding_model)

In [ ]:
retriever_multi_vector_img = create_multi_vector_retriever(
    vectorstore,
    text_summary,
    NarrativeText,
    table_summaries,
    Table,
    image_summaries,
    img_base64_list,
)

In [ ]:
retriever_multi_vector_img

#### Cosine Similarity Search:

In [ ]:
query = "Why We combine a pre-trained retriever (Query Encoder + Document Index) with a pre-trained seq2seq model (Generator) and fine-tune end-to-end?"

In [ ]:
retriever_multi_vector_img.get_relevant_documents(query)

In [ ]:
retriever_multi_vector_img.invoke(query)

#### Above this code we had Vector Store

#### Below code for the Image Data Processing:

In [ ]:
import io
import re
from IPython.display import HTML, display
from PIL import Image

In [ ]:
def is_image_data(b64data):
    image_signatures = {
        b"\xFF\xD8\xFF": "jpg",
        b"\x89\x50\x4E\x47\x0D\x0A\x1A\x0A": "png",
        b"\x47\x49\x46\x38": "gif",
        b"\x52\x49\x46\x46": "webp",
    }
    try:
        header = base64.b64decode(b64data)[:8]  # Decode and get the first 8 bytes
        for sig, format in image_signatures.items():
            if header.startswith(sig):
                return True
        return False
    except Exception:
        return False

In [ ]:
def looks_like_base64(sb):
  return re.match("^[A-Za-z0-9+/]+[=]{0,2}$", sb) is not None


In [ ]:
def resize_base64_image(base64_string,size=(128,128)):
  # Decode the Base64 string
    img_data = base64.b64decode(base64_string)
    img = Image.open(io.BytesIO(img_data))

    # Resize the image
    resized_img = img.resize(size, Image.LANCZOS)

    # Save the resized image to a bytes buffer
    buffered = io.BytesIO()
    resized_img.save(buffered, format=img.format)

    # Encode the resized image to Base64
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


In [ ]:
def split_image_text_types(docs):
    """
    Split base64-encoded images and texts
    """
    b64_images = []
    texts = []

    for doc in docs:
        # Check if the document is of type Document and extract page_content if so
        if isinstance(doc, Document):
            doc = doc.page_content
        if looks_like_base64(doc) and is_image_data(doc):
            doc = resize_base64_image(doc, size=(1300, 600))
            b64_images.append(doc)
        else:
            texts.append(doc)

    return {"images": b64_images, "texts": texts}

In [ ]:
def img_prompt_func(data_dict):
  """
  Join the context into a single string
  """
  print(data_dict)
  formatted_texts = "\n".join(data_dict["context"]["texts"])
  messages = []

  # Adding image(s) to the messages if present
  if data_dict["context"]["images"]:
      for image in data_dict["context"]["images"]:
          image_message = {
              "type": "image_url",
              "image_url": {"url": f"data:image/jpeg;base64,{image}"},
          }
          messages.append(image_message)

  # Adding the text for analysis
  text_message = {
      "type": "text",
      "text": (
          "You are a helpful assistant.\n"
          "You will be given a mixed info(s) .\n"
          "Use this information to provide relevant information to the user question. \n"
          f"User-provided question: {data_dict['question']}\n\n"
          "Text and / or tables:\n"
          f"{formatted_texts}"
      ),
  }
  messages.append(text_message)
  return [HumanMessage(content=messages)]


# DATA GENERATION using Chaining:

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

In [ ]:
def multi_modal_rag_chain(retriever):
  chain=(
      {"context":retriever | RunnableLambda(split_image_text_types),
       "question": RunnablePassthrough()
      }
      | RunnableLambda(img_prompt_func)

      |model
      |StrOutputParser()
  )

  return chain

In [ ]:
chain_multimodal_rag = multi_modal_rag_chain(retriever_multi_vector_img)

In [ ]:
query1="Explain any images / figures in the paper with Left: NQ performance as more documents are retrieved. Center: Retrieval recall performance\
in NQ. Right: MS-MARCO Bleu-1 and Rouge-L as more documents are retrieved."

In [ ]:
chain_multimodal_rag.invoke(query1)

In [ ]:
query = "Why We combine a pre-trained retriever (Query Encoder + Document Index) with a pre-trained seq2seq model (Generator) and fine-tune end-to-end?"


In [ ]:
chain_multimodal_rag.invoke(query)